<center>
<img src='https://i.postimg.cc/9XYKq1nD/movie-theatre.png' width=700>
</center>

# 1. Introduction

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#E32F1E; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #d51e0f;">1.1 Overview</p>

Welcome to this notebook on **sentiment analysis** of IMDb **movie reviews**. The task is to classify whether any given review has a **positive or negative** sentiment. 

We will cover the following:
* *Tokenization using spaCy*
* *Pre-processing (Stop word removal, Lemmatization, etc)*
* *Vectorization using Bag-of-Words and TF-IDF*
* *Modelling via Naive Bayes and Logistic Regression*

<br>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#E32F1E; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #d51e0f;">1.2 Libraries</p>

In [ ]:
# Core
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap
import seaborn as sns
sns.set(style="whitegrid", rc={"axes.edgecolor":"#383838", "grid.color": "#808080"}, font_scale=1.6)
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
from bs4 import BeautifulSoup
import re

# sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score, classification_report, ConfusionMatrixDisplay
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# Color scheme
my_colors = ["#F5DA7F", "#EDBA63", "#E48E4B", "#DE5A36", "#E32F1E", "#CD1C0F"]
CMAP1 = ListedColormap(my_colors)
print("Notebook Color Scheme:")
sns.palplot(sns.color_palette(my_colors))
plt.show()

# 2. EDA

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#E32F1E; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #d51e0f;">2.1 Load data</p>

In [ ]:
# Load data into a pandas data frame
data = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

# Preview data frame and shape
print(data.shape)
data.head()

In [ ]:
# Print first review
print("Sentiment:", data.loc[0,"sentiment"])
print("\nReview:", data.loc[0,"review"])

*Notes:*
* The target is **binary** (positive or negative).
* There are **50,000 reviews** in the dataset.
* The text needs **cleaning** to **remove html tags**.

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#E32F1E; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #d51e0f;">2.2 Distributions</p>

In [ ]:
plt.figure(figsize=(10,4))
sns.countplot(data=data, x="sentiment", palette = [my_colors[1],my_colors[4]])
plt.title("Sentiment distribution")
plt.show()

In [ ]:
# Print target distribution
data["sentiment"].value_counts()

In [ ]:
# Calculate length of each review
len_data = pd.concat([data.loc[:,"review"].map(lambda x:len(x)),data.loc[:,"sentiment"]],axis=1)

# Plot distribution
plt.figure(figsize=(12,5))
sns.histplot(data=len_data, x="review", hue="sentiment", palette = [my_colors[1],my_colors[4]], binwidth=100, kde=True)
plt.title("Length of reviews")
plt.xlabel("Length")
plt.xlim([0,6000])
plt.show()

In [ ]:
print("Average positive review length:", len_data.loc[len_data["sentiment"]=="positive","review"].mean())
print("Average negative review length:", len_data.loc[len_data["sentiment"]=="negative","review"].mean())

*Notes:*
* The target is completely **balanced** (50% positive, 50% negative).
* Negative reviews tend to be marginally **longer** on average.

# 3. Pre-processing

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#E32F1E; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #d51e0f;">3.1 Clean data</p>

Let's clean the data by **removing html tags**, which can be done by using the **BeautifulSoup library**. We'll also **remove urls** and **special characters**.

In [ ]:
def remove_html_tags(text):
    soup = BeautifulSoup(text,"html.parser")
    return soup.get_text()

def remove_urls(text):
    return re.sub("http\S+","",text)

def remove_special_characters(text):
    return re.sub("[^A-Za-z0-9 ]+","",text)

def clean_text(text):
    text = remove_html_tags(text)
    text = remove_urls(text)
    text = remove_special_characters(text)
    return text

In [ ]:
# Data cleaning example
example = " Kaggle<br /><br /> was first launched in 2010. (Website here->[https://www.kaggle.com/])"
print("Before:", example)
print("After:", clean_text(example))

In [ ]:
%%time

# Clean entire dataset
data["review"] = data["review"].apply(clean_text)

Compare this now cleaned **first review**, with the uncleaned version at the top of the notebook.

In [ ]:
# Print first review after data cleaning
data.loc[0,"review"]

Note: It is ok that we have **removed punctuation** since we won't be using sequential models. Instead we are only interested in the **frequency** and relative frequency of words between documents.

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#E32F1E; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #d51e0f;">3.2 Encode target</p>

We need to convert the target to **integers**. We map positive to 1 and negative to 0.

In [ ]:
data["sentiment"] = data["sentiment"].replace("positive",1).replace("negative",0)

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#E32F1E; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #d51e0f;">3.3 Split data</p>

We need to create a **test set** to be able to **evaluate our models**.

In [ ]:
# Split features and labels
X = data["review"]
y = data["sentiment"]

# Split train and test set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0,stratify=y)

# Print train target distribution
print(y_train.value_counts())

This method ensures that the train and test sets **remain balanced**.

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#E32F1E; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #d51e0f;">3.4 Tokenization</p>

We're going to build a custom tokenizer using **spacy**. We'll **remove stop words** (i.e. common words with little meaning) and **apply lemmatization** (i.e. convert words to dictionary form), which are common NLP preprocessing techniques. Furthermore, we only keep tokens if they are an **adjective, proper noun or verb** to reduce the size of the vocabulary.

In [ ]:
# Load English language model
nlp = spacy.load("en_core_web_sm")

# Disable named-entity recognition and parsing to save time
unwanted_pipes = ["ner", "parser"]

# Custom tokenizer using spacy
def custom_tokenizer(doc):
    with nlp.disable_pipes(*unwanted_pipes):
        return [t.lemma_ for t in nlp(doc) if not t.is_stop and not t.is_space and t.pos_ in ["ADJ","NOUN","VERB"]]

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#E32F1E; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #d51e0f;">3.5 Vectorization</p>

We have to convert **variable length strings** of text to **fixed length numerical vectors** to able to train machine learning models on this data. There are two popular approaches for this.

1. **Bow-of-Words**; simply counts how many times each word appears in a document.

In [ ]:
%%time

# Bag-of-Words vectorizer
vectorizer = CountVectorizer(tokenizer=custom_tokenizer)

# Fit and transform train data
X_train_bow = vectorizer.fit_transform(X_train)

# Transform test data
X_test_bow = vectorizer.transform(X_test)

# Print vocab size
print("BoW vocabulary size:", X_train_bow.shape[1])

2. **TF-IDF**; multiples term frequency with inverse document frequency. This puts more weight on words that don't appear in many documents. 

In [ ]:
%%time

# TF-IDF vectorizer
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer)

# Fit and transform train data
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform test data
X_test_tfidf = vectorizer.transform(X_test)

# Print vocab size
print("TF-IDF vocabulary size:", X_train_tfidf.shape[1])

# 4. Modelling with Naive Bayes

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#E32F1E; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #d51e0f;">4.1 Using BoW vectors</p>

Our first model will be **Naive Bayes** on the **Bag-of-Words (BoW) vectorized data**. We'll also tune the additive smoothing hyperparameter `alpha` using grid search. 

In [ ]:
%%time

# Parameters to tune
grid = {"alpha": [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]}

# Define model
clf = MultinomialNB()

# Define grid search
nb_bow = GridSearchCV(clf, param_grid=grid, scoring="f1_macro", n_jobs=-1, cv=5, verbose=5)

# Train model using grid search
nb_bow.fit(X_train_bow, y_train)

# Print best value of alpha
print("Best parameters:", nb_bow.best_params_)

In [ ]:
# Make predictions on test set
nb_bow_preds = nb_bow.predict(X_test_bow)

# Print classification report
print(classification_report(y_test, nb_bow_preds, target_names=["negative","positive"]))

In [ ]:
# Figure size
fig, ax = plt.subplots(figsize=(6, 6))
ax.grid(False)

# Create the confusion matrix
disp = ConfusionMatrixDisplay.from_estimator(nb_bow, X_test_bow, y_test, cmap="Reds", display_labels=["negative","positive"], xticks_rotation="vertical", ax=ax)

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#E32F1E; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #d51e0f;">4.2 Using TF-IDF vectors</p>

Now we'll apply **Naive Bayes** on the **TF-IDF vectorized data**. Let's see how it compares to using Bag-of-Words.

In [ ]:
%%time

# Parameters to tune
grid = {"alpha": [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]}

# Define model
clf = MultinomialNB()

# Define grid search
nb_tfidf = GridSearchCV(clf, param_grid=grid, scoring="f1_macro", n_jobs=-1, cv=5, verbose=5)

# Train model using grid search
nb_tfidf.fit(X_train_tfidf, y_train)

# Print best value of alpha
print("Best parameters:", nb_tfidf.best_params_)

In [ ]:
# Make predictions on test set
nb_tfidf_preds = nb_tfidf.predict(X_test_tfidf)

# Print classification report
print(classification_report(y_test, nb_tfidf_preds, target_names=["negative","positive"]))

In [ ]:
# Figure size
fig, ax = plt.subplots(figsize=(6, 6))
ax.grid(False)

# Create the confusion matrix
disp = ConfusionMatrixDisplay.from_estimator(nb_tfidf, X_test_tfidf, y_test, cmap="Reds", display_labels=["negative","positive"], xticks_rotation="vertical", ax=ax)

# 5. Modelling with Logistic Regression

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#E32F1E; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #d51e0f;">5.1 Using BoW vectors</p>

Now we'll use a **Logistic Regression** model on the **Bag-of-Words vectorized data** to see how it compares to Naive Bayes. We'll also tune the regularization hyperparameters of the model.

In [ ]:
%%time

# Parameters to tune
grid = {"penalty": ["l1","l2"], "solver": ["liblinear"], "C": [0.25, 0.5, 0.75, 1, 1.25, 1.5]}

# Define model
clf = LogisticRegression()

# Define grid search
lr_bow = GridSearchCV(clf, param_grid=grid, scoring="f1_macro", n_jobs=-1, cv=5, verbose=5)

# Train model using grid search
lr_bow.fit(X_train_bow, y_train)

# Print best value of alpha
print("Best parameters:", lr_bow.best_params_)

In [ ]:
# Make predictions on test set
lr_bow_preds = lr_bow.predict(X_test_bow)

# Print classification report
print(classification_report(y_test, lr_bow_preds, target_names=["negative","positive"]))

In [ ]:
# Figure size
fig, ax = plt.subplots(figsize=(6, 6))
ax.grid(False)

# Create the confusion matrix
disp = ConfusionMatrixDisplay.from_estimator(lr_bow, X_test_bow, y_test, cmap="Reds", display_labels=["negative","positive"], xticks_rotation="vertical", ax=ax)

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing: 1px; color:#E32F1E; font-size:100%; text-align:left;padding: 0px; border-bottom: 3px solid #d51e0f;">5.2 Using TF-IDF vectors</p>

Finally, let's use **Logistic Regression** on the **TF-IDF vectorized data**.

In [ ]:
%%time

# Parameters to tune
grid = {"penalty": ["l1","l2"], "solver": ["liblinear"], "C": [0.25, 0.5, 0.75, 1, 1.25, 1.5]}

# Define model
clf = LogisticRegression()

# Define grid search
lr_tfidf = GridSearchCV(clf, param_grid=grid, scoring="f1_macro", n_jobs=-1, cv=5, verbose=5)

# Train model using grid search
lr_tfidf.fit(X_train_tfidf, y_train)

# Print best value of alpha
print("Best parameters:", lr_tfidf.best_params_)

In [ ]:
# Make predictions on test set
lr_tfidf_preds = lr_tfidf.predict(X_test_tfidf)

# Print classification report
print(classification_report(y_test, lr_tfidf_preds, target_names=["negative","positive"]))

In [ ]:
# Figure size
fig, ax = plt.subplots(figsize=(6, 6))
ax.grid(False)

# Create the confusion matrix
disp = ConfusionMatrixDisplay.from_estimator(lr_tfidf, X_test_tfidf, y_test, cmap="Reds", display_labels=["negative","positive"], xticks_rotation="vertical", ax=ax)

# 6. Conclusion

We can summarize the findings as follows:

* TF-IDF did **marginally better** than Bag-of-Words; although the difference was less than 1% in F-1 score. 
* Logistic Regression **outperformed** Naive Bayes by at least 2% in F-1 score in both cases. 
* Logistic Regression may have performed better because of its ability to **regularize** on the large vocabulary.

**References:**
* [Sentiment Analysis of IMDB Movie Reviews](https://www.kaggle.com/code/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews)
* [NLP5 - Text Classification with Naive Bayes](https://www.kaggle.com/code/samuelcortinhas/nlp5-text-classification-with-naive-bayes)

In [ ]:
Thanks for reading!